In [2]:
import com_func

Dataset = "pubmed"

# parameters
threshold = 10
cutoff = 3

coauthor_emb_type = "off"
venue_emb_type = "tf_idf"
year_emb_type = "tf_idf"
pp_textual_emb_type = "off"
citation_emb_type = "off"

In [3]:
def dummy(doc):
    return doc
def read_labeled_file(infile):
    LabeledRecords_original = []
    with open(infile, 'r', encoding = 'utf8') as f:
        for line in f:
            read_data = line.split("\t")
            # get ride of bad formated lines
            if(len(read_data)==13 or len(read_data)==12):
                paper_detail = {"paperID": read_data[0], "authorID":read_data[1], 
                                "co-author": read_data[5], "venue_id": read_data[7],
                                "publish_year": read_data[10]}
                LabeledRecords_original.append(paper_detail)
            else:
                print(len(read_data))
        f.close()
    return pd.DataFrame(LabeledRecords_original)

In [4]:
def LSA(cleaned_token, dim=100):
    # Tf-idf Transformation
    tfidf_vectorizer = TfidfVectorizer(use_idf=True, smooth_idf=True, sublinear_tf=True, tokenizer=dummy,
                                               preprocessor=dummy, stop_words = None,min_df=cutoff)
    tfidfMatrix = tfidf_vectorizer.fit_transform(cleaned_token).toarray()
    if(tfidfMatrix.shape[1]<dim):
        dim = tfidfMatrix.shape[1] -1
    # tf-idf + svd
    svd = TruncatedSVD(n_components=dim)
    final_lsa_Matrix = svd.fit_transform(tfidfMatrix)
    print(svd.explained_variance_ratio_.sum())
    return final_lsa_Matrix

In [5]:
# co-author relation to frequence count
def co_author_to_vector(raw_co_author_data, emb_type="off"):
    while True:
        if emb_type == "tf":
            co_author_vectorizer = CountVectorizer()
            print(co_author_vectorizer)
            result_vector = co_author_vectorizer.fit_transform(raw_co_author_data).toarray()
            #print(co_author_vectorizer.get_feature_names())
            #print(len(co_author_vectorizer.vocabulary_))
            break
        elif emb_type == "tf_idf":
            tfidf_vectorizer = TfidfVectorizer(use_idf=True, smooth_idf=True, sublinear_tf=True,
                                               stop_words = None)
            print(tfidf_vectorizer)
            result_vector = tfidf_vectorizer.fit_transform(raw_co_author_data).toarray()
            break
        elif emb_type == "off":
            result_vector = pd.DataFrame()
            break
        else:
            print("Embedding type not available, selecting default setting")
            emb_type="off"
    return result_vector

In [11]:
# venue relation with author
def venue_to_vector(raw_venue_id, emb_type="off"):
    while True:
        if emb_type == "tf":
            venue_count_vectorizer = CountVectorizer()
            print(venue_count_vectorizer)
            result_vector = venue_count_vectorizer.fit_transform(raw_venue_id).toarray()
            #print(len(venue_count_vectorizer.vocabulary_))
            break
        elif emb_type == "tf_idf":
            tfidf_vectorizer = TfidfVectorizer(use_idf=True, smooth_idf=True, sublinear_tf=True,
                                               stop_words = None)
            print(tfidf_vectorizer)
            result_vector = tfidf_vectorizer.fit_transform(raw_venue_id).toarray()
            break
        elif emb_type == "off":
            result_vector = pd.DataFrame()
            break
        else:
            print("Embedding type not available, selecting default setting")
            emb_type="off"
    return result_vector

In [12]:
# author-year relation to emb
def year_to_vector(raw_year, emb_type="off"):
    while True:
        if emb_type == "tf":
            count_vectorizer = CountVectorizer()
            result_vector = count_vectorizer.fit_transform(raw_year).toarray()
            #print(len(count_vectorizer.vocabulary_))
            break
        elif emb_type == "tf_idf":
            tfidf_vectorizer = TfidfVectorizer(use_idf=True, smooth_idf=True, sublinear_tf=True,
                                               stop_words = None)
            print(tfidf_vectorizer)
            result_vector = tfidf_vectorizer.fit_transform(raw_year).toarray()
            break
        elif emb_type == "off":
            result_vector = pd.DataFrame()
            break
        else:
            print("Embedding type not available, selecting default setting")
            emb_type="tf"
    return result_vector

In [8]:
# document relation wrt textual content
# convert raw text to numerical feature vectors
# bow(Bags of words) are used with uni-gram setting
def raw_text_to_vector(raw_textual_content, emb_type="off", stopword=True):
    cleaned_token, sample_size= com_func.clean_batch_of_raw(raw_textual_content, stopword=stopword)
    average_sample_size = sum(sample_size)/len(sample_size)
    print("Minimal sample size: ", min(sample_size))
    print("maximal sample size: ", max(sample_size))
    while True:
        if emb_type == "tf_idf":
            # using tf-idf
            tfidf_vectorizer = TfidfVectorizer(use_idf=True, smooth_idf=True, sublinear_tf=True, tokenizer=dummy,
                                               preprocessor=dummy, stop_words = None,min_df=cutoff)
            print(tfidf_vectorizer)
            result_vector = tfidf_vectorizer.fit_transform(cleaned_token).toarray()
            #print(len(tfidf_vectorizer.vocabulary_))
            #print(tfidf_vectorizer.get_feature_names())
            break
        elif emb_type == "tf":
            # Document-Term frequence Matrix
            count_vectorizer = CountVectorizer(tokenizer=dummy,preprocessor=dummy, min_df=cutoff)
            result_vector = count_vectorizer.fit_transform(cleaned_token).toarray()
            break
        elif emb_type == "lsa":
            # use lsa
            result_vector = LSA(cleaned_token, dim=100)
            break
        elif emb_type == "off":
            result_vector = pd.DataFrame()
            break
        else:
            print("Embedding type not available, selecting default setting")
            emb_type="off"
    return result_vector, average_sample_size

In [9]:
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score,f1_score,accuracy_score
# cross validation
def k_fold_cv(data, label, clf, k=10):
    kf = KFold(n_splits=k, shuffle=False)
    allTrueLabel = []
    allPredLabel = []
    for train_index, test_index in kf.split(data):
        # print("TRAIN:", train_index, " \n TEST:", test_index)
        # split train and test
        data_train, data_test = data[train_index], data[test_index]
        label_train, label_test = label[train_index], label[test_index]
        # fit data to clf
        clf.fit(data_train, label_train)
        # get predicted label
        label_pred = clf.predict(data_test)
        allTrueLabel.extend(label_test)
        allPredLabel.extend(label_pred)
        # print("True positive: {tp}, False positive: {fp}, False negative: {fn}, True negative: {tn}"
        # .format(tp=round_tp, fp=round_fp, fn=round_fn, tn=round_tn))

    accuracy = accuracy_score(allTrueLabel, allPredLabel)
    f1 = f1_score(allTrueLabel, allPredLabel,average='macro')
    
    print(metrics.classification_report(allTrueLabel, allPredLabel))
    print(metrics.confusion_matrix(allTrueLabel, allPredLabel).ravel())
    
    return accuracy, f1

In [15]:
# load the file
import sys
import io
import os
import collections
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score


fileDir = "../Data/"+Dataset+"/canopies_labeled/"
listfiles = os.listdir(fileDir)
# collect statistic to output
allname = []
num_class = []
per_class_count = []
average_textual_size = []

all_mnb_accuracy = []
all_mnb_f1 = []
all_svcLinear_accuracy = []
all_svcLinear_f1 = []
all_LR_accuracy = []
all_LR_f1 = []

# read all file in labeled group
for file in listfiles:
    # group name
    temp = file.split("_")
    name = temp[1]+"_"+temp[-1]
    print("For name: ",name)
    allname.append(name)
    # read needed content in labeled file
    labeled_data = read_labeled_file(fileDir+file)
    print("total sample size before apply threshold: ",len(labeled_data))
    # count number of paper each author write based on author ID
    paperCounter = collections.Counter(labeled_data["authorID"])
    print(paperCounter)
    # collect per class statistic
    for k in list(paperCounter):
        if paperCounter[k] < threshold:
            del paperCounter[k]
    temp =list(paperCounter.keys())
    print(temp)
    per_class_count.append(paperCounter)
    num_class.append(len(paperCounter))
    # remove samples that are smaller than threshold
    labeled_data = labeled_data[labeled_data.authorID.isin(temp)]
    print("Total sample size after apply threshold: ",len(labeled_data))
    # if only have one class or no class pass the threshold, not applicable
    if(len(paperCounter)==0) or (len(paperCounter)==1):
        average_textual_size.append("Not applicable")
        all_mnb_accuracy.append("Not applicable")
        all_mnb_f1.append("Not applicable")
        all_svcLinear_accuracy.append("Not applicable")
        all_svcLinear_f1.append("Not applicable")
        all_LR_accuracy.append("Not applicable")
        all_LR_f1.append("Not applicable")
    else:
        # convert author id to label
        gather_label = []
        for index, record in labeled_data.iterrows():
            gather_label.append(temp.index(record["authorID"]))
        labeled_data["label"] = gather_label
        # shuffle the data
        labeled_data = labeled_data.sample(frac=1).reset_index(drop=True)
        # extract true label and pid
        label = labeled_data["label"]
        pid = labeled_data["paperID"]
        # list of different data field
        part_collection = []
        # data part 1, co-author matrix
        data_part_co_author = co_author_to_vector(labeled_data["co-author"], emb_type=coauthor_emb_type)
        print(data_part_co_author.shape)
        part_collection.append(data_part_co_author)
        # data part 2.1, venue_id that author attend
        data_part_venue = venue_to_vector(labeled_data["venue_id"], emb_type=venue_emb_type)
        print(data_part_venue.shape)
        part_collection.append(data_part_venue)
        # data part 2.2 year that author attend
        data_part_year = year_to_vector(labeled_data["publish_year"], emb_type=year_emb_type)
        print(data_part_year.shape)
        part_collection.append(data_part_year)
        # merge different part of data data together by concatenate it all together
        # remove empty emb (when emb set off)
        part_collection = [part for part in part_collection if len(part)!=0]
        print(len(part_collection))
        if len(part_collection)>1:
            combinedata = np.concatenate(part_collection,axis=1)
        elif len(part_collection)==1:
            if isinstance(part_collection[0], pd.DataFrame):
                combinedata = part_collection[0].values
            else:
                combinedata = part_collection[0]
        else:
            print("No data available")
            break
        print(combinedata.shape)
        # using converted feature vector to train classifier
        # using Multinomial naive bayes
        clf = MultinomialNB()
        # use 10 fold cv
        mnbaccuracy, mnbmarcof1 = k_fold_cv(combinedata, label, clf, k=10)
        print("MNB Accuracy: ",mnbaccuracy)
        print("MNB F1: ", mnbmarcof1)
        all_mnb_accuracy.append(mnbaccuracy)
        all_mnb_f1.append(mnbmarcof1)
        # using SVM with linear kernal
        clf = SVC(decision_function_shape='ovr', kernel='linear')
        svcaccuracy, svcmarcof1 = k_fold_cv(combinedata, label, clf, k=10)
        print("svc Accuracy: ",svcaccuracy)
        print("svc F1: ", svcmarcof1)
        all_svcLinear_accuracy.append(svcaccuracy)
        all_svcLinear_f1.append(svcmarcof1)
        # using logistic regression
        clf = LogisticRegression(multi_class='ovr')
        LRaccuracy, LRmarcof1 = k_fold_cv(combinedata, label, clf, k=10)
        print("LR Accuracy: ",LRaccuracy)
        print("LR F1: ", LRmarcof1)
        all_LR_accuracy.append(LRaccuracy)
        all_LR_f1.append(LRmarcof1)
    
# write evaluation result to excel
output = pd.DataFrame({'Name Group':allname,"Class number":num_class,"per_class_size":per_class_count, 
                       "svc(linear) accuracy":all_svcLinear_accuracy, "svc(linear) macro f1": all_svcLinear_f1, 
                       "mnb accuracy":all_mnb_accuracy, "mnb macro f1": all_mnb_f1,
                       "logistic regression accuracy":all_LR_accuracy, "logistic regression macro f1": all_LR_f1})

savePath = "../result/"+Dataset+"/year_venue_only/"
if not os.path.exists(savePath):
    os.makedirs(savePath)
filename = "2004_meta_"+venue_emb_type+"_threshold="+str(threshold)+".csv"
output.to_csv(savePath+filename, encoding='utf-8',index=False)
print("Done")

For name:  j_read
total sample size before apply threshold:  136
Counter({'0000-0002-5159-1192': 57, '0000-0002-9029-5185': 39, '0000-0002-9697-0962': 31, '0000-0002-4739-9245': 3, '0000-0003-0605-5259': 3, '0000-0003-4316-7006': 1, '0000-0002-0784-0091': 1, '0000-0002-3888-6631': 1})
['0000-0002-9697-0962', '0000-0002-9029-5185', '0000-0002-5159-1192']
Total sample size after apply threshold:  127
(0, 0)
TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=True,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)
(127, 79)
TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='conte

/home/gao137/intel/intelpython3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
/home/gao137/intel/intelpython3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


             precision    recall  f1-score   support

          0       0.56      0.29      0.38        31
          1       0.85      0.59      0.70        39
          2       0.61      0.89      0.72        57

avg / total       0.67      0.65      0.63       127

[ 9  2 20  3 23 13  4  2 51]
LR Accuracy:  0.6535433070866141
LR F1:  0.6011175585643671
For name:  f_esteves
total sample size before apply threshold:  34
Counter({'0000-0002-3046-1313': 18, '0000-0002-5403-0091': 12, '0000-0003-0589-0746': 3, '0000-0003-3172-6253': 1})
['0000-0002-5403-0091', '0000-0002-3046-1313']
Total sample size after apply threshold:  30
(0, 0)
TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=True,
        token_p

/home/gao137/intel/intelpython3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
/home/gao137/intel/intelpython3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


             precision    recall  f1-score   support

          0       0.70      0.58      0.64        12
          1       0.75      0.83      0.79        18

avg / total       0.73      0.73      0.73        30

[ 7  5  3 15]
svc Accuracy:  0.7333333333333333
svc F1:  0.7129186602870814
             precision    recall  f1-score   support

          0       0.71      0.42      0.53        12
          1       0.70      0.89      0.78        18

avg / total       0.70      0.70      0.68        30

[ 5  7  2 16]
LR Accuracy:  0.7
LR F1:  0.6534017971758665
For name:  c_miller
total sample size before apply threshold:  252
Counter({'0000-0003-4341-1283': 51, '0000-0002-3989-7973': 40, '0000-0002-3813-1706': 39, '0000-0003-2772-9531': 27, '0000-0001-6082-9273': 22, '0000-0002-2601-4422': 22, '0000-0002-9448-8144': 19, '0000-0001-8628-4902': 15, '0000-0002-2936-7717': 6, '0000-0003-3898-9734': 6, '0000-0002-5074-6914': 2, '0000-0003-4266-6700': 1, '0000-0002-9286-9787': 1, '0000-0002-08

...

(0, 0)
TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=True,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)
(65, 36)
TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=True,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)
(65, 20)
2
(65, 56)
             precision    recall  f1-score   support

          0       0.92

/home/gao137/intel/intelpython3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
/home/gao137/intel/intelpython3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
/home/gao137/intel/intelpython3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.is

             precision    recall  f1-score   support

          0       0.60      0.60      0.60        15
          1       0.62      0.62      0.62        16

avg / total       0.61      0.61      0.61        31

[ 9  6  6 10]
LR Accuracy:  0.6129032258064516
LR F1:  0.6125
For name:  c_adams
total sample size before apply threshold:  69
Counter({'0000-0003-2100-4417': 43, '0000-0001-5602-2741': 20, '0000-0002-7333-9908': 4, '0000-0003-1628-4020': 1, '0000-0002-0667-8088': 1})
['0000-0003-2100-4417', '0000-0001-5602-2741']
Total sample size after apply threshold:  63
(0, 0)
TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=True,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True

/home/gao137/intel/intelpython3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
/home/gao137/intel/intelpython3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
/home/gao137/intel/intelpython3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.is

             precision    recall  f1-score   support

          0       0.88      0.64      0.74        11
          1       0.70      0.97      0.81        31
          2       1.00      0.10      0.18        10

avg / total       0.79      0.73      0.67        52

[ 7  4  0  1 30  0  0  9  1]
LR Accuracy:  0.7307692307692307
LR F1:  0.5764903659640502
Done


In [16]:
# accuracy
from statistics import mean 
cleaned_mnb_accuracy = [x for x in all_mnb_accuracy if isinstance(x, float)]
cleaned_svcLinear_accuracy = [x for x in all_svcLinear_accuracy if isinstance(x, float)]
cleaned_lr_accuracy = [x for x in all_LR_accuracy if isinstance(x, float)]
print(len(cleaned_mnb_accuracy))
print(len(cleaned_svcLinear_accuracy))
print(len(cleaned_lr_accuracy))
print(mean(cleaned_mnb_accuracy))
print(mean(cleaned_svcLinear_accuracy))
print(mean(cleaned_lr_accuracy))

784
784
784
0.7192512241904999
0.77393245727873
0.7259381668203747


In [17]:
# f1
from statistics import mean 
# remove string from result
cleaned_mnb_f1 = [x for x in all_mnb_f1 if isinstance(x, float)]
cleaned_svcLinear_f1 = [x for x in all_svcLinear_f1 if isinstance(x, float)]
cleaned_lr_f1 = [x for x in all_LR_f1 if isinstance(x, float)]
print(len(cleaned_mnb_f1))
print(len(cleaned_svcLinear_f1))
print(len(cleaned_lr_f1))
print(mean(cleaned_mnb_f1))
print(mean(cleaned_svcLinear_f1))
print(mean(cleaned_lr_f1))

784
784
784
0.598677461782394
0.7096296153254906
0.5971807362010636


In [ ]:
%reset

In [ ]:
%who